In [1]:
"""from google.colab import drive
drive.mount('/content/gdrive/')

%cd /content/gdrive/MyDrive/ThermostabilityProject/
!pip install transformers tqdm"""

"from google.colab import drive\ndrive.mount('/content/gdrive/')\n\n%cd /content/gdrive/MyDrive/ThermostabilityProject/\n!pip install transformers tqdm"

In [2]:
import pandas as pd
import numpy as np
import torch, torch.nn
from transformers import BertModel, BertTokenizer
import re

import re, gc, os
import requests
from tqdm.auto import tqdm

/Users/kvr226/miniconda3/envs/latent_msaprot_env/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
def train_test_validation_splits(df):
    test,train_tot = df.loc[df['set']=='test'],df.loc[df['set']=='train']
    train, val = train_tot.loc[train_tot['validation']!=True], train_tot.loc[train_tot['validation']==True]

    
    return train,val, test

def embed_dataset(dataset_seqs, model, tokenizer, device, shift_left = 1, shift_right = -1):
  inputs_embedding = []

  for sample in tqdm(dataset_seqs):
    with torch.no_grad():
      ids = tokenizer.batch_encode_plus([sample], add_special_tokens=True, padding=True, is_split_into_words=True, return_tensors="pt")
      embedding = model(input_ids=ids['input_ids'].to(device))[0]
      inputs_embedding.append(embedding[0].detach().cpu().numpy()[shift_left:shift_right])

  return torch.from_numpy(np.stack(inputs_embedding))

In [4]:
path_meltome = 'datasets/FLIP/splits/meltome/splits/mixed_split.csv'

splits_meltome = pd.read_csv(path_meltome, sep=',')
train, val, test = train_test_validation_splits(splits_meltome)
print('done')

done


In [5]:
tokenizer = BertTokenizer.from_pretrained("Rostlab/prot_bert", do_lower_case=False )
model = BertModel.from_pretrained("Rostlab/prot_bert")
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

model = model.to(device=device)
model = model.eval()

Some weights of the model checkpoint at Rostlab/prot_bert were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [6]:

dir_pre_generated_embeddings = ''
path_train= dir_pre_generated_embeddings+'train_set_embeddings.pth'
path_test= dir_pre_generated_embeddings+'test_set_embeddings.pth'
path_val= dir_pre_generated_embeddings+'val_set_embeddings.pth'



if os.path.isfile(path_train) and os.path.isfile(path_test) and os.path.isfile(path_val):
    print ("Loading Premade Embeddings...")
    train_embeddings = torch.load(path_train, map_location=device)
    test_embeddings = torch.load(path_test, map_location=device)
    val_embeddings = torch.load(path_val, map_location=device)
    print ("Loaded")
else:
    train_embeddings = embed_dataset(train.sequence, model, tokenizer, device)
    test_embeddings = embed_dataset(test.sequence, model, tokenizer, device)
    val_embeddings = embed_dataset(val.sequence, model, tokenizer, device)
    torch.save(train_embeddings, 'train_set_embeddings.pth')
    torch.save(test_embeddings, 'test_set_embeddings.pth')
    torch.save(val_embeddings, 'val_set_embeddings.pth')

Loading Premade Embeddings...
Loaded


In [7]:
import torch.utils.data as data_utils

batch_size=200


trainDataset = data_utils.TensorDataset(train_embeddings, torch.from_numpy( train.target.to_numpy() ))
testDataset = data_utils.TensorDataset(test_embeddings, torch.from_numpy( test.target.to_numpy() ))
valDataset = data_utils.TensorDataset(val_embeddings, torch.from_numpy( val.target.to_numpy() ))

train_loader = data_utils.DataLoader(trainDataset, batch_size=batch_size, shuffle=True)
test_loader = data_utils.DataLoader(testDataset, batch_size=batch_size, shuffle=True)
val_loader = data_utils.DataLoader(valDataset, batch_size=batch_size, shuffle=True)

In [8]:
class Flatten(torch.nn.Module):
    def __init__(self):
        super(Flatten, self).__init__()
    
    def forward(self, input):
        return input.view(input.size(0), -1)   

class regressionHead(torch.nn.Module):

    def __init__(self, kernel):
        super(regressionHead, self).__init__()
        #self.input_shape = input_shape # pos0 = #channels, pos1 = #diagonal comps, or viseversa

        self.pooling = torch.nn.Sequential(
            torch.nn.AvgPool1d(kernel, stride=2, padding=kernel//2),
            Flatten())

        self.FFNN = torch.nn.Sequential(
            torch.nn.Linear(512, 512),
            torch.nn.LeakyReLU(0.1), #nn.ReLU(),
            torch.nn.Linear(512, 512),
            torch.nn.LeakyReLU(0.1), #nn.ReLU(),
            torch.nn.Linear(512, 1)
        )
        
        #self.encoder.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, torch.nn.Linear):
            torch.nn.init.kaiming_normal_(module.weight)

    def forward(self, x):
        x = self.pooling(x)
        z = self.FFNN(x)
        return z #z.permute(0,2,1) 

In [9]:
def train_LLMRegresor(train_iterator, val_iterator, model, device, criterion, optimizer, epoch_num):
    
        
        val_loss = []
        
        model.to(device)

        for epoch in range(epoch_num):
            model.train() 
            for i, (input, label) in enumerate(train_iterator):
                optimizer.zero_grad()
                
                input = input.to(dtype=torch.float32, device=device)
                label = label.to(dtype=torch.float32, device=device) 
                out = model(input)
                loss = criterion(out, label.unsqueeze(-1))
                loss.backward() 
                optimizer.step()

            with torch.no_grad(): # evaluate validation loss here 

                model.eval()
                val_loss_epochs = []

                for (inputval, labelval) in val_iterator:
                    
                    inputval = inputval.to(device)
                    labelval = labelval.to(device)
                    outval = model(inputval)
                    lossval = criterion(outval, labelval.unsqueeze(-1)) # Calculate validation loss 
                    val_loss_epochs.append(lossval.item())

                val_loss_epoch = np.mean(val_loss_epochs)
                val_loss.append(round(val_loss_epoch, 3))

            print('epoch: %d loss: %.3f val loss: %.3f' % (epoch + 1, loss.item(), val_loss_epoch))

            '''# evalutate whether validation loss is dropping; if not, stop
            if epoch > 21:
                if val_loss[-1] >= np.min(val_loss[:-20]): 
                    print('Finished training at epoch {0}'.format(epoch))
                    return epoch'''

In [10]:
model = regressionHead(kernel=3)
opt = torch.optim.AdamW(model.parameters(), lr=1e-5)
loss= torch.nn.MSELoss()
epochs = 3500
train_LLMRegresor(train_loader, val_loader, model, device, loss, opt, epochs)

epoch: 1 loss: 2909.012 val loss: 2762.943
epoch: 2 loss: 2715.153 val loss: 2722.006
epoch: 3 loss: 2624.065 val loss: 2635.666
epoch: 4 loss: 2592.348 val loss: 2514.049
epoch: 5 loss: 2269.957 val loss: 2322.357
epoch: 6 loss: 2175.083 val loss: 2095.253
epoch: 7 loss: 1767.318 val loss: 1825.978
epoch: 8 loss: 1376.160 val loss: 1509.756
epoch: 9 loss: 1405.571 val loss: 1212.661
epoch: 10 loss: 879.089 val loss: 923.968
epoch: 11 loss: 555.846 val loss: 669.342
epoch: 12 loss: 519.346 val loss: 469.327
epoch: 13 loss: 364.412 val loss: 320.379
epoch: 14 loss: 158.262 val loss: 221.962
epoch: 15 loss: 147.460 val loss: 161.804
epoch: 16 loss: 115.980 val loss: 133.168
epoch: 17 loss: 113.839 val loss: 121.008
epoch: 18 loss: 125.127 val loss: 116.824
epoch: 19 loss: 91.354 val loss: 114.968
epoch: 20 loss: 135.085 val loss: 114.476
epoch: 21 loss: 113.085 val loss: 113.057
epoch: 22 loss: 99.349 val loss: 111.923
epoch: 23 loss: 71.572 val loss: 112.447
epoch: 24 loss: 95.032 val l

In [13]:
test_set, test_labels = val_loader.dataset.tensors

In [14]:
test_set = test_set.to(dtype=torch.float32, device=device)
test_labels = test_labels.to(dtype=torch.float32, device=device) 
outcome = model(test_set)
loss_test = loss(outcome, test_labels.unsqueeze(-1))
print(loss_test)

tensor(112.8791, grad_fn=<MseLossBackward0>)


In [11]:



'''sequence_Example = "A E T C Z A O"
sequence_Example = re.sub(r"[UZOB]", "X", sequence_Example)
encoded_input = tokenizer(sequence_Example, return_tensors='pt')
output = model(**encoded_input)
print(output)'''

'sequence_Example = "A E T C Z A O"\nsequence_Example = re.sub(r"[UZOB]", "X", sequence_Example)\nencoded_input = tokenizer(sequence_Example, return_tensors=\'pt\')\noutput = model(**encoded_input)\nprint(output)'